In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os
#import json
import torch

import time

from tqdm import tqdm

In [3]:
###setting directory
data_dir="data"
model_dir="models"
output_dir="output"
###

# Finetune model: load dataset

In [4]:
#change
from datasets import Dataset
#PVE_input = Dataset.from_pandas(pd.read_csv('data\PVE_covid_annotation_v0_1.csv', encoding='latin1',index_col=0))
PVE_input =pd.read_excel('data\PVE_covid_annotation_v2_annotated.xlsx', index_col=0)
train_set=Dataset.from_pandas(PVE_input[(PVE_input['BL_batch']>=0) & (PVE_input['BL_batch']<=2)])
test_set=Dataset.from_pandas(PVE_input[PVE_input['BL_batch']==-1])

In [5]:
train_set

Dataset({
    features: ['opinion', 'project', 'random_order', 'BL_batch', 'AL_batch', 'Autonomy_for_companies', 'Autonomy_for_close_contact_professions', 'Quality_of_life_for_elderly', 'Family_value', 'Feasibility', 'Acceptance_of_exception', 'The_need_of_schooling', 'Working_safety', 'Well-being', 'Reduce_adverse_effect', 'step_0_margin', 'step_0_rank', 'step_1_margin', 'step_1_rank', 'step_2_margin', 'step_2_rank', 'step_3_margin', 'step_3_rank', 'step_4_margin', 'step_4_rank', 'step_5_margin', 'step_5_rank', 'step_6_margin', 'step_6_rank', 'step_7_margin', 'step_7_rank', 'step_8_margin', 'step_8_rank', 'step_9_margin', 'step_9_rank', '__index_level_0__'],
    num_rows: 150
})

In [6]:
labels = ['Autonomy_for_companies',
 'Autonomy_for_close_contact_professions',
 'Quality_of_life_for_elderly',
 'Family_value',
 'Feasibility',
 'Acceptance_of_exception',
 'The_need_of_schooling',
 'Working_safety',
 'Well-being',
 'Reduce_adverse_effect']

In [7]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [8]:
label2id 

{'Autonomy_for_companies': 0,
 'Autonomy_for_close_contact_professions': 1,
 'Quality_of_life_for_elderly': 2,
 'Family_value': 3,
 'Feasibility': 4,
 'Acceptance_of_exception': 5,
 'The_need_of_schooling': 6,
 'Working_safety': 7,
 'Well-being': 8,
 'Reduce_adverse_effect': 9}

# Preprocess data

In [9]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    #text = examples["Tweet"]
    # encode them
    encoding = tokenizer(examples['project'], examples['opinion'], truncation=True)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(examples['opinion']), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()
  
    return encoding


In [10]:
encoded_train_set = train_set.map(preprocess_data, batched=True, remove_columns=train_set.column_names)
encoded_test_set = test_set.map(preprocess_data, batched=True, remove_columns=test_set.column_names)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
example =encoded_train_set[0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [12]:
tokenizer.decode(example['input_ids'])

'[CLS] nursing homes allow visitors again [SEP] loneliness in the last phase of life is inhumane and also leads to premature death. [SEP]'

In [13]:
example['labels']

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Quality_of_life_for_elderly', 'Well-being']

In [15]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    encoded_train_set.set_format("torch", device='cuda:0')
    encoded_test_set.set_format("torch", device='cuda:0')
else:
    encoded_train_set.set_format("torch")
    encoded_test_set.set_format("torch")
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Define model

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./models/bert_train_level1/' , 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert_train_level1/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([54, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([54]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    model.to('cuda')
else:    
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Train the model

In [18]:
batch_size = 8
metric_name = "f1"

In [19]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    #evaluation_strategy = "epoch",
    #save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    #learning_rate=2e-5,
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #num_train_epochs=40,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=25,
    #push_to_hub=True,
)

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [21]:
encoded_train_set[0]['labels'].type()

'torch.cuda.FloatTensor'

In [22]:
encoded_train_set['input_ids'][0]

tensor([  101,  8329,  5014,  3499,  5731,  2153,   102, 20334,  1999,  1996,
         2197,  4403,  1997,  2166,  2003, 29582,  2063,  1998,  2036,  5260,
         2000, 21371,  2331,  1012,   102], device='cuda:0')

In [23]:
encoded_train_set['input_ids'][0].device

device(type='cuda', index=0)

In [24]:
#forward pass
outputs = model(input_ids=encoded_train_set['input_ids'][0].unsqueeze(0), labels=encoded_train_set['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7643, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1438,  0.4147, -0.0165, -0.0189,  0.6883,  0.4039,  0.0585, -0.0507,
         -0.0621, -0.2267]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_set,
    eval_dataset=encoded_test_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

***** Running training *****
  Num examples = 150
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 380
  Number of trainable parameters = 109489930
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
25,0.397900,0.283290,0.527273,0.683904,0.270000
50,0.289300,0.259555,0.588957,0.723305,0.250000
75,0.276900,0.244294,0.573196,0.714866,0.280000
100,0.253500,0.258629,0.569072,0.712942,0.300000
125,0.222200,0.258933,0.605469,0.737676,0.260000
150,0.213100,0.280118,0.550388,0.711483,0.250000
175,0.192100,0.243140,0.636042,0.769828,0.245000
200,0.175300,0.227279,0.675325,0.781650,0.310000
225,0.142500,0.222704,0.720137,0.823566,0.350000
250,0.120800,0.235142,0.702341,0.818098,0.350000


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  N

TrainOutput(global_step=380, training_loss=0.17385635226964952, metrics={'train_runtime': 1912.9764, 'train_samples_per_second': 1.568, 'train_steps_per_second': 0.199, 'total_flos': 159457780159608.0, 'train_loss': 0.17385635226964952, 'epoch': 20.0})

# Evaluate

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


{'eval_loss': 0.21926139295101166,
 'eval_f1': 0.7272727272727274,
 'eval_roc_auc': 0.8308228363196992,
 'eval_accuracy': 0.4,
 'eval_runtime': 18.5406,
 'eval_samples_per_second': 10.787,
 'eval_steps_per_second': 1.348,
 'epoch': 20.0}

# Inference

In [28]:
test_pred=trainer.predict(encoded_test_set).predictions
test_pred = torch.from_numpy(test_pred)

***** Running Prediction *****
  Num examples = 200
  Batch size = 8


In [29]:
pd.DataFrame(1*(test_pred.sigmoid()>0.5)).sum(axis=0)

0    72
1    37
2    32
3     5
4     3
5    32
6     5
7     6
8    73
9    22
dtype: int64

In [30]:
from sklearn.metrics import  classification_report

print(classification_report(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:15],np.matrix(1*(test_pred.sigmoid()>0.5))))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91        64
           1       0.95      0.85      0.90        41
           2       0.62      0.95      0.75        21
           3       0.60      0.21      0.32        14
           4       0.67      0.14      0.24        14
           5       0.66      0.64      0.65        33
           6       0.40      0.50      0.44         4
           7       0.50      0.19      0.27        16
           8       0.71      0.78      0.74        67
           9       0.73      0.48      0.58        33

   micro avg       0.75      0.70      0.73       307
   macro avg       0.67      0.57      0.58       307
weighted avg       0.74      0.70      0.70       307
 samples avg       0.71      0.66      0.66       307



In [31]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:15],np.matrix(1*(test_pred.sigmoid()>0.5)))

array([[[126,  10],
        [  2,  62]],

       [[157,   2],
        [  6,  35]],

       [[167,  12],
        [  1,  20]],

       [[184,   2],
        [ 11,   3]],

       [[185,   1],
        [ 12,   2]],

       [[156,  11],
        [ 12,  21]],

       [[193,   3],
        [  2,   2]],

       [[181,   3],
        [ 13,   3]],

       [[112,  21],
        [ 15,  52]],

       [[161,   6],
        [ 17,  16]]], dtype=int64)

In [32]:
model.save_pretrained('models')

Configuration saved in models\config.json
Model weights saved in models\pytorch_model.bin
